<a href="https://colab.research.google.com/github/nukano0522/time_series_analytics/blob/main/prophet/prophet_with_multiprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import pandas as pd
import numpy as np
from prophet import Prophet

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
print(df.shape)
df.head()

(2905, 2)


,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


In [5]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/zgsw34gj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/1szxfeph.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=60091', 'data', 'file=/tmp/tmpltfe9as5/zgsw34gj.json', 'init=/tmp/tmpltfe9as5/1szxfeph.json', 'output', 'file=/tmp/tmpltfe9as5/prophet_modele344v_fu/prophet_model-20221106000443.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:04:43 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:04:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [8]:
forecast = m.predict(df)
print(forecast.shape)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

(2905, 19)


,ds,yhat,yhat_lower,yhat_upper
2900,2016-01-16,8.228240,7.617133,8.819282
2901,2016-01-17,8.609055,8.026071,9.221236
2902,2016-01-18,8.932837,8.372303,9.569864
2903,2016-01-19,8.718778,8.058112,9.294284
2904,2016-01-20,8.549661,7.920518,9.172105


In [13]:
ts_table = [df for _ in range(100)]

# 直列に処理を繰り返す

In [15]:
from time import time
from tqdm import tqdm

all_fit_time = 0
all_forecast_time = 0

for i in tqdm(range(100)):
  m = Prophet()

  start = time()
  m.fit(ts_table[i])
  fit_time = time() - start
  
  start = time()
  forecast = m.predict(ts_table[i])
  forecast_time = time() - start

  all_fit_time += fit_time
  all_forecast_time += forecast_time

print(f"学習にかかった時間: {all_fit_time}")
print(f"予測にかかった時間: {all_forecast_time}")

  0%|          | 0/100 [00:00<?, ?it/s]INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/ebv6stce.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/l5dkxyer.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97301', 'data', 'file=/tmp/tmpltfe9as5/ebv6stce.json', 'init=/tmp/tmpltfe9as5/l5dkxyer.json', 'output', 'file=/tmp/tmpltfe9as5/prophet_modeljzly1tvu/prophet_model-20221106001457.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:14:57 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:14:59 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
  1%|          | 1/100 [00:02<04:13,  2.56s/it]INFO:prophet:Disabling daily seasonality. Run prophet wit

学習にかかった時間: 127.06384015083313
予測にかかった時間: 135.9484851360321


# 並列処理

In [18]:
import multiprocessing
from multiprocessing import Pool

In [30]:
# 利用できるコア数
cpu_core_num = multiprocessing.cpu_count()
print(cpu_core_num)

2


In [36]:
class ProphetWithMultiprocessing:

  def __init__(self, dataset, cpu_core_num):
    self.dataset = dataset
    self.cpu_core_num = cpu_core_num
    self.work_table = [pd.DataFrame() for _ in range(100)] # 中間結果の格納用
    self.output_table = np.nan # 最終結果の格納用


  def prophet_fitting_and_forecast(self, idx):
    m = Prophet()
    m.fit(self.dataset[idx])
    forecast = m.predict(self.dataset[idx])
    return forecast


  def calc_prophet_forecast(self, idx):
    self.work_table[idx] = self.prophet_fitting_and_forecast(idx)
    return self.work_table[idx]


  def run(self):
    idxs = [idx for idx in range(100)]

    start = time()
    with Pool(processes=self.cpu_core_num) as pool:
      self.output_table = pool.map(self.calc_prophet_forecast, idxs)

    processing_time = time() - start

    print(f"処理時間: {processing_time}")


In [37]:
pwm = ProphetWithMultiprocessing(dataset=ts_table, cpu_core_num=cpu_core_num)
pwm.run()
    

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/zutgtmsj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/07ewkowh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/6_5tddug.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpltfe9as5/0xhstz_2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=17683', 'data', 'file=/tmp/tmpltfe9as5/zutgtmsj.json', 'init=/tmp/tmpltfe9as5/0xhstz_2.json', 'output', 'file=/tmp/tmpltfe9as5/prophet_modelyxydohp1/prophet_model-20221106003639.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
DEBUG:cmdstanpy:CmdStan args: ['/usr/loc

処理時間: 204.97185969352722


In [45]:
pwm.output_table[0]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2007-12-10,8.035820,8.196087,9.438761,8.035820,8.035820,0.802814,0.802814,0.802814,0.352279,0.352279,0.352279,0.450535,0.450535,0.450535,0.0,0.0,0.0,8.838634
1,2007-12-11,8.034297,7.939024,9.216262,8.034297,8.034297,0.552915,0.552915,0.552915,0.119616,0.119616,0.119616,0.433299,0.433299,0.433299,0.0,0.0,0.0,8.587212
2,2007-12-12,8.032774,7.746369,9.001991,8.032774,8.032774,0.350351,0.350351,0.350351,-0.066648,-0.066648,-0.066648,0.416999,0.416999,0.416999,0.0,0.0,0.0,8.383125
3,2007-12-13,8.031250,7.718772,8.987710,8.031250,8.031250,0.329798,0.329798,0.329798,-0.072279,-0.072279,-0.072279,0.402077,0.402077,0.402077,0.0,0.0,0.0,8.361048
4,2007-12-14,8.029727,7.734074,8.927864,8.029727,8.029727,0.319379,0.319379,0.319379,-0.069561,-0.069561,-0.069561,0.388940,0.388940,0.388940,0.0,0.0,0.0,8.349107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900,2016-01-16,7.564528,7.585119,8.874261,7.564528,7.564528,0.663712,0.663712,0.663712,-0.311712,-0.311712,-0.311712,0.975424,0.975424,0.975424,0.0,0.0,0.0,8.228240
2901,2016-01-17,7.563503,8.026291,9.264287,7.563503,7.563503,1.045552,1.045552,1.045552,0.048305,0.048305,0.048305,0.997247,0.997247,0.997247,0.0,0.0,0.0,8.609055
2902,2016-01-18,7.562477,8.284015,9.524148,7.562477,7.562477,1.370360,1.370360,1.370360,0.352279,0.352279,0.352279,1.018080,1.018080,1.018080,0.0,0.0,0.0,8.932837
2903,2016-01-19,7.561452,8.126611,9.350711,7.561452,7.561452,1.157326,1.157326,1.157326,0.119616,0.119616,0.119616,1.037711,1.037711,1.037711,0.0,0.0,0.0,8.718778
